<a href="https://colab.research.google.com/github/karinayang454/Twitter_Sentiment_Analysis/blob/main/TwitterSentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
# change to correct directory
%cd /content/drive/My Drive/CAIS/WinterProject

Mounted at /content/drive/
/content/drive/My Drive/CAIS/WinterProject


In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
# imports
import re
import matplotlib.pyplot as plt
import string
from nltk.corpus import stopwords
import nltk
nltk.download("wordnet")
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from collections import Counter
from wordcloud import WordCloud
from nltk.corpus import stopwords
import nltk
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
import gensim
from sklearn.model_selection import train_test_split
import spacy
import pickle
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
import matplotlib.pyplot as plt 
import tensorflow as tf
import keras
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Embedding
from keras import layers
from keras.optimizers import *
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
# read in data
STORE_PATH = '/content/drive/MyDrive/CAIS/WinterProject/'
df=pd.read_csv(STORE_PATH + 'data.csv')
df = df[['tweet','valence']] # drop column: "author"
df.head(10)


,tweet,valence
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0
1,is upset that he can't update his Facebook by ...,0
2,@Kenichan I dived many times for the ball. Man...,0
3,my whole body feels itchy and like its on fire,0
4,"@nationwideclass no, it's not behaving at all....",0
5,@Kwesidei not the whole crew,0
6,Need a hug,0
7,@LOLTrish hey long time no see! Yes.. Rains a...,0
8,@Tatiana_K nope they didn't have it,0
9,@twittera que me muera ?,0


In [ ]:
# few notes about the dataset
# df['valence'].unique()               0 and 4
# len(df)                              1.6 million
# df.groupby('valence').count()        0: 8 million, 4: 8 million
# df.groupby('valence').nunique()        #about 1-3k repeats

In [ ]:
# function to clean the data
def depure_data(data): 
    # Removing URLs with a regular expression
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    data = url_pattern.sub(r'', data)
    # Remove Emails
    data = re.sub('\S*@\S*\s?', '', data)
    # Remove new line characters
    data = re.sub('\s+', ' ', data)
    # # Remove distracting single quotes
    data = re.sub("\'", "", data)
        
    return data


# further preprocess the tweets
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence),     deacc=True))

# detokenize text
def detokenize(text):
    return TreebankWordDetokenizer().detokenize(text)

In [ ]:
temp = []
data_to_list = df['tweet'].values.tolist()
for i in range(len(data_to_list)):
    temp.append(depure_data(data_to_list[i]))
data_words = list(sent_to_words(temp))
print(data_words[:10])

In [ ]:
data = []
for i in range(len(data_words)):
    data.append(detokenize(data_words[i]))
print(data[:10])
longest_string=max(data,key=len)
print(len(longest_string)) # 193 is max length

['awww that bummer you shoulda got david carr of third day to do it', 'is upset that he can update his facebook by texting it and might cry as result school today also blah', 'dived many times for the ball managed to save the rest go out of bounds', 'my whole body feels itchy and like its on fire', 'no it not behaving at all mad why am here because can see you all over there', 'not the whole crew', 'need hug', 'hey long time no see yes rains bit only bit lol fine thanks how you', 'nope they didn have it', 'que me muera']
193


In [1]:
data = np.array(data)
# convert range of values from {0,4} to {0,1}
labels = np.array(df['valence']//4)
len(labels)

NameError: ignored

In [ ]:
max_words = 7000
max_len = 195

# convert tweets to word embeddings
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data)
sequences = tokenizer.texts_to_sequences(data)
tweets = pad_sequences(sequences, maxlen=max_len)
print(tweets)

[[   0    0    0 ...    1   38    4]
 [   0    0    0 ...   37  256 1092]
 [   0    0    0 ...   35   30   10]
 ...
 [   0    0    0 ...   13    9 1988]
 [   0    0    0 ...   10 4954   48]
 [   0    0    0 ...    0    0  113]]


In [ ]:
# split data into train and test
X_train, X_test, y_train, y_test = train_test_split(tweets,labels, train_size=0.6,random_state=0)
print (len(X_train),len(X_test),len(y_train),len(y_test))

960000 640000 960000 640000


In [ ]:
# Bidirectional model
model = Sequential()
model.add(layers.Embedding(max_words, 40, input_length=max_len))
model.add(layers.Bidirectional(layers.LSTM(20,return_sequences=True, input_shape=(None,40),dropout=0.3)))
model.add(layers.Bidirectional(layers.LSTM(40,dropout=0.6)))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['binary_accuracy'])
history = model.fit(X_train, y_train, epochs=10, batch_size=2048, validation_data=(X_test, y_test),shuffle=True)

Epoch 1/10
469/469 [==============================] - 203s 413ms/step - loss: 0.4797 - binary_accuracy: 0.7675 - val_loss: 0.4375 - val_binary_accuracy: 0.7974
Epoch 2/10
469/469 [==============================] - 192s 410ms/step - loss: 0.4345 - binary_accuracy: 0.7976 - val_loss: 0.4250 - val_binary_accuracy: 0.8024
Epoch 3/10
469/469 [==============================] - 192s 409ms/step - loss: 0.4234 - binary_accuracy: 0.8033 - val_loss: 0.4186 - val_binary_accuracy: 0.8062
Epoch 4/10
469/469 [==============================] - 192s 409ms/step - loss: 0.4158 - binary_accuracy: 0.8076 - val_loss: 0.4150 - val_binary_accuracy: 0.8091
Epoch 5/10
469/469 [==============================] - 191s 408ms/step - loss: 0.4102 - binary_accuracy: 0.8111 - val_loss: 0.4115 - val_binary_accuracy: 0.8115
Epoch 6/10
469/469 [==============================] - 191s 408ms/step - loss: 0.4052 - binary_accuracy: 0.8139 - val_loss: 0.4097 - val_binary_accuracy: 0.8120
Epoch 7/10
469/469 [====================